In [1]:
using DataFrames, StatFiles
using Statistics

In [2]:
# Import dataset as dataframe
df = load("NEW7080.dta") |> DataFrame;

In [3]:
indx = @. (df[:, 27] >= 30) & (df[:, 27] <= 39)
df = df[indx, :]
educ = df[:, 4]
lwklywge = df[:, 9];

In [6]:
struct myIVBound
    px
    Eyx
    unq_x

    function myIVBound(y, x)
        # Data parameters
        n = length(y)
        unq_x = sort(unique(x))
        px = zeros(length(unq_x))
        Eyx = zeros(length(unq_x))
        for u in 1:length(unq_x)
            # Estimate P(x = u) and E[y|x = u]
            indx = x .== unq_x[u]
            px[u] = sum(indx) / n
            Eyx[u] = mean(y[indx])
        end
        new(px, Eyx, unq_x)
    end #MYIVBOUND
end #MYIVBOUND

In [8]:
function coef(fit::myIVBound, s, t)
    Eyx_px = fit.Eyx .* fit.px

    from_t = fit.unq_x .> t
    until_s = fit.unq_x .< s
    
    ub = sum(Eyx_px[from_t]) + (fit.Eyx[fit.unq_x .== t] * sum(fit.px[.!from_t]))[1]
    lb = sum(Eyx_px[until_s]) + (fit.Eyx[fit.unq_x .== s] * sum(fit.px[.!until_s]))[1]
    
    return ub - lb
end

coef (generic function with 1 method)

In [9]:
fit = myIVBound(lwklywge, educ);

In [10]:
coef(fit, 12, 16)/4

0.11202714900306754

In [11]:
bounds = zeros(19)
for j in 2:20
    bounds[j - 1] = coef(fit, j - 1, j)
end

In [12]:
bounds

19-element Vector{Float64}:
 0.8361493989101385
 0.7339395208060582
 0.728347306811429
 0.6384001837934452
 0.5740888677438534
 0.5042198905181223
 0.4448289522360005
 0.35468933004610737
 0.2935308064467135
 0.2684520422704866
 0.2564983532992233
 0.21343974487222006
 0.21454698115353832
 0.23170190633874288
 0.3916546646499208
 0.3369873794462892
 0.37239451417343794
 0.3153741336578584
 0.4144109499808719